In [1]:
import json
from pathlib import Path
from typing import List

import PIL
import PIL.Image
import torchvision.transforms.functional as F
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize


import json
import multiprocessing
from argparse import ArgumentParser
from datetime import datetime
from pathlib import Path
from typing import List
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utils import base_path, ShoesDataset, FashionIQDataset, targetpad_transform, CIRRDataset,squarepad_transform
from utils import collate_fn, update_train_running_results, set_train_bar_description, save_model, \
    extract_index_features, generate_randomized_fiq_caption, device

D:\SoftWare\Anaconda\envs\fa_clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_path_list = []

lines_train = [] # 用于保存每一行字符串的数组
lines_test = [] # 用于保存每一行字符串的数组

base_path = "E://Code//data//shoesDataset//"


file_path_train = base_path + "data//shoes//shoes-cap-train.txt"
file_path_test = base_path + "data//shoes//shoes-cap-test.txt"

with open(file_path_test, 'r') as file:
    for line in file:
        temp_list = line.strip().split(';')
        lines_test.append(temp_list)  # strip()用于移除行末尾的换行符和空白字符
        image_path_list.append(temp_list[0])
        image_path_list.append(temp_list[1])

with open(file_path_train, 'r') as file:
    for line in file:
        temp_list = line.strip().split(';')
        lines_train.append(temp_list)  # strip()用于移除行末尾的换行符和空白字符
        image_path_list.append(temp_list[0])
        image_path_list.append(temp_list[1])

In [3]:
len(image_path_list)

42892

In [4]:
len(lines_train),len(lines_test)

(17954, 3492)

In [5]:
image_path_list = list(set(image_path_list))

In [6]:
len(image_path_list)

10776

In [7]:
base_path + image_path_list[0]

'E://Code//data//shoesDataset//data/shoes/attributedata/womens_rain_boots/0/img_womens_rain_boots_269.jpg'

In [8]:
preprocess = targetpad_transform(1.25, 288)

D:\SoftWare\Anaconda\envs\fa_clip\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [9]:
A = preprocess(PIL.Image.open(base_path + image_path_list[0]))

In [10]:
A.shape

torch.Size([3, 288, 288])

In [2]:
preprocess = targetpad_transform(1.25, 288)
classic_val_dataset = ShoesDataset('val', 'classic', preprocess)
relative_val_dataset = ShoesDataset('test', 'relative', preprocess)
relative_train_dataset = ShoesDataset('train', 'relative', preprocess)

Shoes val dataset in classic mode initialized
Shoes test dataset in relative mode initialized
Shoes train dataset in relative mode initialized


D:\SoftWare\Anaconda\envs\fa_clip\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [3]:
classic_val_dataset.__len__(), relative_val_dataset.__len__(), relative_train_dataset.__len__()

(10776, 3492, 17954)

In [8]:
classic_val_loader = DataLoader(dataset=classic_val_dataset, batch_size=32, num_workers=4,
                                pin_memory=True, collate_fn=collate_fn)

relative_val_loader = DataLoader(dataset=relative_val_dataset, batch_size=32, num_workers=4,
                                pin_memory=True, collate_fn=collate_fn)

relative_train_loader = DataLoader(dataset=relative_train_dataset, batch_size=32, num_workers=4,
                                pin_memory=True, collate_fn=collate_fn)

In [9]:
_, (image_name_A, image_A) = next(enumerate(classic_val_loader))

In [11]:
len(image_name_A), image_name_A[0], image_A.shape

(32,
 'data/shoes/attributedata/womens_stiletto/1/img_womens_stiletto_1580.jpg',
 torch.Size([32, 3, 288, 288]))

In [12]:
_, (reference_name_B, target_name_B, image_captions_B)  = next(enumerate(relative_val_loader))

In [20]:
len(image_captions_B),reference_name_B.shape, target_name_B.shape

(32, torch.Size([32, 3, 288, 288]), torch.Size([32, 3, 288, 288]))

In [14]:
_, (reference_image_C, target_image_C, image_captions_C)  = next(enumerate(relative_train_loader))

In [21]:
reference_image_C.shape, target_image_C.shape, len(image_captions_C), len(image_captions_C)

(torch.Size([32, 3, 288, 288]), torch.Size([32, 3, 288, 288]), 32, 32)

In [22]:
image_captions_C

['are teal suede with closed toes',
 'are suede, with a buckle',
 'are shiny maroon with no straps',
 'have clear plastic uppers',
 'are brown leather sandals',
 'are essentially identical',
 'have straps on the side',
 'have laces and brown accents',
 'are a tennis shoe with two velcro closures',
 'is shiny blue, not black',
 'has a squared toe',
 'are pink and white, with no laces',
 'are silver, not white',
 'has a wider heel and is brown',
 'have a strap and a thicker heel',
 'have two straps',
 'are slip on shoes',
 'are silver',
 'are gold, not silver',
 'are matte, with a closed toe',
 'are black with gold embellishments',
 'are black with a white paisley print',
 'are black with some red',
 'is black with laces',
 'are solid black',
 'has a thinner heel and straps',
 'are pale metallic, not black',
 'is black with a thinner, higher heel',
 'has a taller heel and thicker buckles',
 'is black, with no thick buckles',
 'are lighter brown with a higher heel',
 'has a higher platfor